In [32]:
# Import
import os
import cv2
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from keras.applications.resnet import preprocess_input
from numpy.linalg import norm # Normalize
from keras.models import load_model
import pickle
from flask import Flask, request, jsonify
from PIL import Image

In [33]:
# =====================
#     FUNCTIONS
# =====================

def recommend(features,feature_list):
    neighbors = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')
    neighbors.fit(feature_list)

    distances, indices = neighbors.kneighbors([features])

    return indices

def extract_feature(img_path, model):
  img = cv2.imread(img_path)
  img = cv2.resize(img, (150,150))
  img = np.array(img)
  expand_img = np.expand_dims(img, axis=0)
  pre_img = preprocess_input(expand_img)
  result = model.predict(pre_img).flatten()
  normalized = result/norm(result)
  return normalized

In [34]:
# =================
# MODELS IMPORT
# =================

model_top_down = load_model("Models/model_top_down.h5")
model_extraction = load_model("Models/feature_extract_model.h5")

top_feature_list = np.array(pickle.load(open("Models/top_feature_extraction.pkl", "rb")))
bottom_feature_list = np.array(pickle.load(open("Models/bottom_feature_extraction.pkl", "rb")))

top_filenames = pickle.load(open('Models/top_directory.pkl', "rb"))
bottom_filenames = pickle.load(open("Models/bottom_directory.pkl", "rb"))

top_filenames_df = pickle.load(open('Models/top_directory_df.pkl', 'rb'))
bottom_filenames_df = pickle.load(open('Models/bottom_directory_df.pkl', 'rb'))

top_url = pickle.load(open("Models/top_url.pkl", "rb"))
bottom_url = pickle.load(open("Models/bottom_url.pkl", "rb"))

df = pd.read_csv("dataset8.csv", index_col=False)

In [71]:
image_path = "Dataset//7476018_m.jpg"

# Image preprocessing
image = Image.open(image_path)
image = image.resize((150, 150))
image = np.array(image) / 255.0
image = np.expand_dims(image, axis=0)

# Predict the category
prediction = model_top_down.predict(image)
predicted_class = 'top' if prediction[0][0] > 0.5 else 'bottom'

print(predicted_class)


1/1 [==============================] - 0s 38ms/step


top


In [86]:
import random

if predicted_class == 'top':
    features = extract_feature(image_path, model_extraction)
    indices = recommend(features, top_feature_list)

    # Recommendation file_path
    target_file = random.choice([top_filenames[indices[0][0]], top_filenames[indices[0][1]], top_filenames[indices[0][2]], top_filenames[indices[0][3]], top_filenames[indices[0][4]]])

    # Ambil setId dari rekomendasi
    filtered_df = df.loc[df['file_path'] == target_file]

    set_id = filtered_df["setId"]

    for set in set_id:
        set_id = set
        break

    print(set_id)

    # Teruskan setId ke dataframe bottom
    recommended_path = bottom_filenames_df[bottom_filenames_df['setId'] == set_id]["file_path"]

    for path in recommended_path:
        recommended_path = path
        break

    print(recommended_path)

    # Recommend the bottom
    features = extract_feature(recommended_path, model_extraction)
    indices = recommend(features, bottom_feature_list)

    # Print the recommendation
    target_files = [
        bottom_filenames[indices[0][0]],
        bottom_filenames[indices[0][1]],
        bottom_filenames[indices[0][2]],
        bottom_filenames[indices[0][3]],
        bottom_filenames[indices[0][4]],
    ]

    price_output = []
    target_link = []

    for file in target_files:
        filtered_df = bottom_filenames_df.loc[bottom_filenames_df['file_path'] == file]
        prices = filtered_df["price"].values
        if len(prices) > 0:
            price_output.append(prices[0])
        else:
            price_output.append('Unknown')
        for target in filtered_df["url"]:
            target=target
            break
        target_link.append(target)

    print(target_link)
    print(price_output)


else:
    features = extract_feature(image_path, model_extraction)
    indices = recommend(features, bottom_feature_list)

    # Recommendation file_path
    target_file = bottom_filenames[indices[0][0]]

    # Ambil setId dari rekomendasi
    filtered_df = df.loc[df['file_path'] == target_file]

    set_id = filtered_df["setId"]

    for set in set_id:
        set_id = set
        break

    # Teruskan setId ke dataframe top
    recommended_path = top_filenames_df[top_filenames_df['setId'] == set_id]["file_path"]

    for path in recommended_path:
        recommended_path = path
        break

    # Recommend the top
    features = extract_feature(recommended_path, model_extraction)
    indices = recommend(features, top_feature_list)

    # Print the recommendation
    target_files = [
        top_filenames[indices[0][0]],
        top_filenames[indices[0][1]],
        top_filenames[indices[0][2]],
        top_filenames[indices[0][3]],
        top_filenames[indices[0][4]],
    ]

    price_output = []
    target_link = []

    for file in target_files:
        filtered_df = top_filenames_df.loc[top_filenames_df['file_path'] == file]
        prices = filtered_df["price"].values
        if len(prices) > 0:
            price_output.append(prices[0])
        else:
            price_output.append('Unknown')
        for target in filtered_df["url"]:
            target=target
            break
        target_link.append(target)

    print(target_link)
    print(price_output)

1/1 [==============================] - 0s 207ms/step
3996102
Dataset/39659948_m.jpg
1/1 [==============================] - 0s 132ms/step
['https://storage.googleapis.com/fashionism_dataset/39659948_m.jpg', 'https://storage.googleapis.com/fashionism_dataset/40314740_m.jpg', 'https://storage.googleapis.com/fashionism_dataset/44584501_m.jpg', 'https://storage.googleapis.com/fashionism_dataset/34139023_m.jpg', 'https://storage.googleapis.com/fashionism_dataset/14236887_m.jpg']
[310911.76470588235, 130435.29411764706, 87979.41176470589, 64711.76470588235, 140517.64705882352]


In [81]:
bottom_filenames_df[bottom_filenames_df['url'] == "https://storage.googleapis.com/fashionism_dataset/40314740_m.jpg"]

,setId,file_path,price,itemName,expressions,category_mapped,url
660,4011823,Dataset/40314740_m.jpg,130435.294118,スキニージーンズ,['アイテム説明-【送料】商品のサイズや重量、配送ルートよって異なる\r\n【配送期間】3～...,Bottom,https://storage.googleapis.com/fashionism_data...
